<a href="https://colab.research.google.com/github/arehfeldt/machine-learning-dump/blob/main/Aaron_Rehfeldt_CSE5522_Naive_Bayes_for_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**CSE 5522 Hands-on #2: Naive Bayes**

The goals of today's exercise are to familarize you with:


*   Naive Bayes
*   Binary Classification
*   Data exploration

**END OF CLASS GOAL:** Submit a link to your notebook (Share > Get Sharable Link) in Carmen so I can see how far you got.  This should be submitted in a group assignment page on Carmen.

**Part 0: Initial setup**

**0.1:** Make a copy of this page in your google drive so that you can edit it. Edit the filename to include your group number.  Share the copied page with your teammates. At the end of class, share a URL and submit (so I can see how far you got).  This will count as the participation grade for all members.

**0.2:** While not completely necessary for this assignment, you may want to familiarize yourself with the following packages: [numpy](https://numpy.org), [scikit-learn](https://scikit-learn.org), [pandas](https://pandas.pydata.org), [matplotlib](https://matplotlib.org).

---
---


**Part 1: A Simple Bayes Net: Naive Bayes**

In class, we discussed how a joint probablity distribution can be broken into individual conditional probabilities. Then, depending on what conditional independences exist, we hoped to simplify those conditional probabilities. In a later lecture, we talked about constructing Bayesian Networks (BN), which gives us a framework for discovering these conditional independences.

For now, though, one of the simplest form of BNs is the Naive Bayes model which encodes a set of simple conditional independences:

- Given a single cause all of the effects are independent from each other.
- Mathematically:
$P($*cause*$, $*effect*$_1, ..., $*effect*$_n) = P($*cause*$) \prod_i P($*effect*$_i|$*cause*$)$

NB can be used for classification by assuming that cause is the true (unknown) label and it (probabilistically) generates all of the features (effects) while features are independent given the cause.

For example, in sentiment analysis the *cause* is the author's sentiment (say, unknown label from the set of {sad, happy, feared, suprised, disgusted, angry}) and the *effects* are words that s/he writes. The simplifying assumption of NB says that knowing the latent sentiment, words of the sentence are independent. We know this assumption is not true because grammar and word-use impose some dependency structure between words in the sentence, but we choose to ignore that in this model.

Although simple, NB has shown good performance in many classifcation tasks and has become a standard classic baseline for classification.

Today we want to perform Twitter sentiment analysis using NB. The goal is to figure out if a tweet has a positive or negative sentiment about the weather.  

**1.0:** Set up the environment (you can click on the play button below to import the appropriate modules).

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

**1.1** Read the data from GitHub into a pandas dataframe.

In [ ]:
TweetUrl='https://github.com/aasiaeet/cse5522data/raw/master/db3_final_clean.csv'
tweet_dataframe=pd.read_csv(TweetUrl)

**1.2** Print out the top of the dataframe to make sure that the data loaded correctly.  It should be a data table with three columns (weight, tweet, label), and 3697 rows.

In [ ]:
display(tweet_dataframe.shape)
tweet_dataframe.head()

(3697, 3)

,weight,tweet,label
0,1.0000,it is very cold out want it to be warmer,-1
1,0.7698,dammmmmmm its pretty cold this morning burr lol,-1
2,0.6146,why does halsey have to be so far away think m...,-1
3,0.9356,dammit stop being so cold so can work out,-1
4,1.0000,its too freakin cold,-1


Labels are -1 and +1 for negative and positive sentiments respectively. Multiple judges have been asked to choose a label for a tweet (this is an example of crowd-sourcing) from five possible labels:

- Tweet is not relevant to weather.
- I can't tell the sentiment.
- Neutral: author just sharing information.
- Positive
- Negative

The majority vote was picked as the label and its ratio was set as the weight of the tweet. So for the tweet in row 2 above, 61% of judges voted that the label is negative.

Note that tweets have been pre-processed (or cleaned). For example, :) and :( :) were replaced with "sad" and "smiley" and numbers with "num", etc. You can go further (as we ask in 1.12) and remove the stop words, i.e., repetitive non-informative words such as am, is, and are.

**1.3.** In the next step, we should build our feature matrix by converting the string of words to a vector of numeric values.

First we need to assign a unique id to each word and create the feature matrix with correct size:

In [ ]:
# wordDict maps words to id
# X is the document-word matrix holding the presence/absence of words in each tweet
wordDict = {}
idCounter = 0
for i in range(tweet_dataframe.shape[0]):
  allWords = tweet_dataframe.iloc[i,1].split(" ")
  for word in allWords:
    if word not in wordDict:
      wordDict[word] = idCounter
      idCounter += 1
X = np.zeros((tweet_dataframe.shape[0], idCounter),dtype='float')

Checking head of the dictionary:

In [ ]:
dict(list(wordDict.items())[0:10])

{'': 9,
 'be': 7,
 'cold': 3,
 'is': 1,
 'it': 0,
 'out': 4,
 'to': 6,
 'very': 2,
 'want': 5,
 'warmer': 8}

**1.4:** The simplest way of coding a tweet to numbers is to mark the occurrence of a word, and forget about its frequency in the document (tweet). This works well with tweets as there are not many repetitive words in a single tweet. So let's fill the document-word matrix:

In [ ]:
for i in range(tweet_dataframe.shape[0]):
  allWords = tweet_dataframe.iloc[i,1].split(" ")
  for word in allWords:
    X[i, wordDict[word]]  = 1

Now we check if the number of words are correct:

In [ ]:
np.sum(X[0:5, ], axis = 1)

array([10.,  9., 17.,  9.,  4.])

Finally, we extract the labels from the dataframe:

In [ ]:
y = np.array(tweet_dataframe.iloc[:,2])
y[0:5]

array([-1, -1, -1, -1, -1])

Let's compute the total number of positive and negative tweets:

In [ ]:
numNeg = np.sum(y < 0)
numPos = np.sum(y > 0) #len(y) - numNeg
probNeg = numNeg / (numNeg + numPos)
probPos = 1 - probNeg
display(numNeg, numPos, probNeg, probPos)

1650

2047

0.4463078171490398

0.5536921828509602

So samples 0:1649 are negative and 1650: are positive.

**1.5: Train/Test Split** Now with do the 20/80 split and learn the word probabilities using the 80 % part and test the NB performance on the 20 % part.

In [ ]:
from sklearn.model_selection import train_test_split
xTrain, xTest, yTrain, yTest = train_test_split(X, y, test_size = 0.2, random_state = 0)
display(xTrain.shape, xTest.shape, yTrain.shape, yTest.shape)

(2957, 5989)

(740, 5989)

(2957,)

(740,)

**1.6: Computing Probabilities by Counting** Now the real work begins. Write the code that, from the train feature matrix xTrain computes the needed word probabilites, i.e., $P(word|label)$ where label is + or - and word is any of the words saved in the `wordDict`:

In [ ]:
# compute three distributions (four variables):
#
# probWordGivenPositive: P(word|Sentiment = +ive)
# probWordGivenNegative: P(word|Sentiment = -ive)
# priorPositive: P(Sentiment = +ive)
# priorNegative: P(Sentiment = -ive)
#  (note these last two form one distribution)

# compute distributions here
probWordGivenPositive = sum(xTrain[yTrain > 0, 0:10]) / sum(yTrain > 0)
probWordGivenNegative = sum(xTrain[yTrain < 0, 0:10]) / sum(yTrain < 0)


priorPositive = 0
priorNegative = 0

# tried to find a way to do this with numpy but couldnt figuree it out so I looped over yTrain instead
for y in yTrain:
  if y > 0:
    priorPositive += 1
  else:
    priorNegative += 1
priorPositive /= yTrain.size
priorNegative /= yTrain.size

# checking the results
display(probWordGivenPositive[0:5])
display(probWordGivenNegative[0:5])
display(priorPositive, priorNegative)

array([0.1185006 , 0.20737606, 0.01088271, 0.01451028, 0.10217654])

array([0.14504988, 0.19493477, 0.00537222, 0.09669992, 0.13967767])

0.5593506932702063

0.4406493067297937

Note that you only needed to compute $P(word = 1| +)$ or $P(word = 1| -)$ and the probabilities of the word being absent from a tweet is just 1 minus those probabilities.

However, as we see in 1.7, for convenience, we will also want to compute $log P(word = 1 | +)$, $log P(word = 0 | +)$, $log P(word = 1 | -)$ and $log P(word = 0 | -)$.  Also we should compute the log priors.  Let's do so now.


In [ ]:
# compute the following:
# logProbWordPresentGivenPositive
logProbWordPresentGivenPositive = np.log(probWordGivenPositive)

# logProbWordAbsentGivenPositive
logProbWordAbsentGivenPositive = np.log(-1 * probWordGivenPositive + 1)

# logProbWordPresentGivenNegative
logProbWordPresentGivenNegative = np.log(probWordGivenNegative)


# logProbWordAbsentGivenNegative
logProbWordAbsentGivenNegative = np.log(-1 * probWordGivenNegative + 1)

# logPriorPositive
logPriorPositive = np.log(priorPositive)


# logPriorNegative
logPriorNegative = np.log(priorNegative)

display(logProbWordPresentGivenPositive)
display(logProbWordPresentGivenNegative)
display(logProbWordAbsentGivenPositive)
display(logProbWordAbsentGivenNegative)

display (logPriorPositive)
display (logPriorNegative)


# Did this work, or did you get an error?  (Read below.)

array([-2.13283722, -1.57322143, -4.52058012, -4.23289805, -2.28105316,
       -4.31990942, -1.44480514, -2.64026725, -5.10836678, -0.20680258])

array([-1.93067756, -1.63509031, -5.22651443, -2.33614267, -1.96841789,
       -3.80512875, -1.35234165, -2.53769559, -4.9752    , -0.28079868])

array([-0.12613096, -0.23240639, -0.01094236, -0.01461658, -0.10778182,
       -0.01339034, -0.2689153 , -0.07401496, -0.0060643 , -1.6776106 ])

array([-0.15671216, -0.21683197, -0.0053867 , -0.10170047, -0.15044815,
       -0.02250774, -0.29926074, -0.08234774, -0.0069311 , -1.40723347])

-0.5809786442688406

-0.8195059427276321

You likely received an error when you tried to take $log(0)$ at some point.  Can your group think of a way to avoid taking $log(0)$?  Check in with your instructor/TA to see if what you're thinking will work.  Implement that change in your code above.

**1.7: Math of NB** Here we provide the derivation of NB when we want to classify the $i$th tweet $\textbf{x}^{(i)}$ and the size of dictionary is $p$, i.e., each tweet is a binary vector of size $p$ as $\textbf{x}^{(i)} = (x_1^{(i)},\dots, x_p^{(i)})$.

Note that we computed $P(x_j^{(i)} = 1|+)$ and $P(x_j^{(i)} = 1|-)$ in above code from `xTrain` and now want to classify `xTest` samples.

**Classification Rule:** For each tweet $i$ NB classifier assigns label + if $P(+|\textbf{x}^{(i)}) > P(-|\textbf{x}^{(i)})$ and negative otherwise.

These posterior probabilities can be computed using prior probabilities (that we got from `xTrain`) and Bayes rule as follows:

\begin{align}
P(+|\textbf{x}^{(i)}) &= \alpha P(\{\textbf{x}^{(i)}\}_{i=1}^n | +)P(+)
\\
(\text{NB Assumption}) &= \alpha P(+) \prod_{j=1}^p P(x_j^{(i)}|+)
\end{align}

For computational convinence (preventing underflow while dealing with small numbers) we work with the $\log$ of probabilities:

\begin{align}
\log(P(+|\textbf{x}^{(i)})) &\propto \log P(+) + \sum_{j=1}^p \log P(x_j^{(i)}|+)
\\
\log(P(-|\textbf{x}^{(i)})) &\propto \log P(-) + \sum_{j=1}^p \log P(x_j^{(i)}|-)
\end{align}

Finally we can compute the confidence of our prediction as the log of the ratio of posteriors:
$\log(\frac{P(\text{predicted label}|\textbf{x}^{(i)})}{P(\text{the other label}|\textbf{x}^{(i)})})$


**1.8: Implementing NB** Now write a function that takes a row of `xTest` and output a label for it based on NB classification rule.


In [ ]:
# classifyNB:
#   words - vector of words of the tweet (binary vector)
#   logProbWordPresentGivenPositive - log P(x_j = 1|+)
#   logProbWordAbsentGivenPositive  - log P(x_j = 0|+)
#   logProbWordPresentGivenNegative - log P(x_j = 1|-)
#   logProbWordAbsentGivenNegative  - log P(x_j = 0|-)
#   logPriorPositive - log P(+)
#   logPriorNegative - log P(-)
#   returns (label of x according to the NB classification rule, confidence about the label)

# Note: you can also change the function definition if you wish to encapsulate all six log probs
# as one model; just make sure to follow through below

def classifyNB(words,logProbWordPresentGivenPositive, logProbWordAbsentGivenPositive,
               logProbWordPresentGivenNegative, logProbWordAbsentGivenNegative,
               logPriorPositive, logPriorNegative):


print(classifyNB(xTest[700, ], logProbWordPresentGivenPositive,logProbWordAbsentGivenPositive,
                               logProbWordPresentGivenNegative, logProbWordAbsentGivenNegative,
                               logPriorPositive, logPriorNegative))

IndentationError: ignored

**1.9:** Compute the output of `classifyNB` for all test data and output the average error.  

In [ ]:
# testNB: Classify all xTest
#   xTest - test data features
#   yTest - true label of test data
#   logProbWordPresentGivenPositive - log P(x_j = 1|+)
#   logProbWordAbsentGivenPositive  - log P(x_j = 0|+)
#   logProbWordPresentGivenNegative - log P(x_j = 1|-)
#   logProbWordAbsentGivenNegative  - log P(x_j = 0|-)
#   logPriorPositive - log P(+)
#   logPriorNegative - log P(-)
#   returns Average test error
def testNB(xTest, yTest,
           logProbWordPresentGivenPositive, logProbWordAbsentGivenPositive,
           logProbWordPresentGivenNegative, logProbWordAbsentGivenNegative,
           logPriorPositive, logPriorNegative):

  # compute avgErr

  print("Average error of NB is", avgErr)
  return avgErr

testNB(xTest, yTest,
       logProbWordPresentGivenPositive, logProbWordAbsentGivenPositive,
       logProbWordPresentGivenNegative, logProbWordAbsentGivenNegative,
       logPriorPositive, logPriorNegative)

**1.10:** Now write an outer wrapper that performs 10 train/test splits and compute the mean and standard deviation of the average accuracy across 10 runs.

In [ ]:
# 10 train/test splits


**1.11:** Compare your results against a known NB algorithm from *scikit-learn*.  Do you get the same average accuracy? (Run the 80/20 split 10 times using the code below as a template)




In [ ]:
from sklearn.naive_bayes import # pick a module here
# fit the model

In [ ]:
# 10 train/test splits using sklearn
